In [ ]:
# this could be a class, instantiated with the user's steam key to connect to the api

In [91]:
import pandas as pd
import numpy as np
import requests
import keyring
from IPython.display import clear_output

In [ ]:
# keyring.set_password('steam',username = 'IcyJoseph',password = 'E0152C2E7BF8A5AA0361A4DADCCE2287') # password config
# keyring.get_password('steam', 'IcyJoseph')

In [ ]:
number_ids_you_want = 1000000 # how many unique steam ids do you want? the API queryer will stop when it has more than
                              # this many

In [78]:
password = keyring.get_password(service_name = 'steam', username = 'IcyJoseph')
steam_id = '76561198985648791' # this is my steam_id
new_id = '76561198020258797' # this is an ID I got from reddit - here:
#                             https://www.reddit.com/r/Steam/comments/68yr8k/building_a_big_list_of_steam_ids_hit_me_up_if_you/

In [3]:
file = open('steam_ids', mode = 'r')# read in file
user_ids = file.read()
user_ids = user_ids.replace(" ","") # cleaning
user_ids = user_ids[1:-1].split(",")
user_ids = pd.DataFrame({'user_id':user_ids})

# user_ids.head()

,user_id
0,151603712
1,59945701
2,53875128
3,26122540
4,126340495


In [6]:
def get_player_info(steam_id):
    
    response = requests.get(
        f'http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={password}&steamids={steam_id}')
    return response

In [14]:
def get_friends(steam_id):
    # returns basic profile information
    response = requests.get(
        f'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={password}&steamid={steam_id}&relationship=friend')
    return response

In [8]:
def get_games(steam_id):
    # returns basic profile information
    response = requests.get(
        f'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={password}&steamid={steam_id}&format=json')
    return response

In [20]:
# me = get_player_info(steam_id)
# friends = get_friends(steam_id)
# games = get_games(steam_id)

In [21]:
# me.content # my profile info

b'{"response":{"players":[{"steamid":"76561198985648791","communityvisibilitystate":3,"personaname":"josephwthomas","lastlogoff":1566641611,"profileurl":"https://steamcommunity.com/profiles/76561198985648791/","avatar":"https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb.jpg","avatarmedium":"https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb_medium.jpg","avatarfull":"https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb_full.jpg","personastate":1,"primaryclanid":"103582791429521408","timecreated":1566556156,"personastateflags":0}]}}'

In [22]:
# friends.content # i have no friends

b'{"friendslist":{"friends":[]}}'

In [23]:
# games.content # i only have one game. need to get a lookup from appid to name of game (it's middle earth: shadow of mordor)

b'{"response":{"game_count":1,"games":[{"appid":241930,"playtime_forever":0}]}}'

In [ ]:
# next steps:
    # get list of active steam_ids (scraping forum?)
    # find a lookup from appid to game name
    # create function that takes friend's steam_id as an input and returns their top (5? 10?) played games
    # pull some sort of info on players

In [80]:
# takes a steam_id and returns all the id's friends
def find_all_friends(steam_id):
    
    friends = get_friends(steam_id) # query steam api

    splitfriends = str(friends.content).split('"')
    all_friends = [i for i in splitfriends if i.isnumeric() == True] # items which are all numeric are steam_ids
    
    return all_friends

In [92]:
# need to get a list of active steam ids. 
# I'm pulling friends own a known popular ID, then pulling friends of each friend, and so on until I get 1M IDs

counter = 0
all_ids = list()
all_ids.append(new_id)

while len(all_ids) < number_ids_you_want:
    
    id_num = all_ids[counter]
    steam_ids = find_all_friends(id_num)
    
    all_ids.extend(steam_ids) # .extend() adds an iterable to a list, returning one list instead of a list of list
    all_ids = set(all_ids)
    all_ids = list(all_ids)
    
#     clear_output()
    print("run "+str(counter+1)+" finished, all_ids is "+str(len(all_ids))+" items long")
    
    counter += 1

run 2197 finished, all_ids is 100056 items long


In [93]:
file = open('all_steam_ids', mode = 'w+') # in the future, the lookup will just be a function i'll import and call
file.write(str(all_ids))

2101176